## Training guide


In [11]:
WANDB_TOKEN=None
!wandb login --relogin $WANDB_TOKEN

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/nhtlong/.netrc


In [1]:
from pkg.models import MODEL_REGISTRY
print(MODEL_REGISTRY)

/home/nhtlong/mambaforge/envs/ucc/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Registry of MODEL:
╒══════════════════════════╤═══════════════════════════════════════════════════════════════════════════╕
│ Names                    │ Objects                                                                   │
╞══════════════════════════╪═══════════════════════════════════════════════════════════════════════════╡
│ SuperviseModel           │ <class 'core.models.base.SuperviseModel'>                                 │
├──────────────────────────┼───────────────────────────────────────────────────────────────────────────┤
│ DeeplabV3ResNet50        │ <class 'pkg.models.supervise.deeplabv3.DeeplabV3ResNet50'>                │
├──────────────────────────┼───────────────────────────────────────────────────────────────────────────┤
│ DeeplabV3ResNet50VicregL │ <class 'pkg.models.supervise.deeplabv3_vicregl.DeeplabV3ResNet50VicregL'> │
├──────────────────────────┼───────────────────────────────────────────────────────────────────────────┤
│ SegResNet50              │ <class 

In [1]:
!CUDA_VISIBLE_DEVICES=1 ucc-train -c ../configs/public/tf/dpt-ade.yml -o global.find_lr=True global.wandb=False data.args.train.root_dir='../data' data.args.val.root_dir='../data' 

Overriding configurating
Seed set to 1940
Some weights of DPTForSemanticSegmentation were not initialized from the model checkpoint at Intel/dpt-large-ade and are newly initialized: ['neck.fusion_stage.layers.0.residual_layer1.batch_norm1.running_mean', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.weight', 'neck.fusion_stage.layers.0.residual_layer1.batch_norm2.running_mean', 'neck.fusion_stage.layers.0.residual_layer1.batch_norm2.bias', 'neck.fusion_stage.layers.0.residual_layer1.batch_norm1.running_var', 'neck.fusion_stage.layers.0.residual_layer1.batch_norm1.weight', 'neck.fusion_stage.layers.0.residual_layer1.batch_norm1.bias', 'neck.fusion_stage.layers.0.residual_layer1.batch_norm2.running_var', 'neck.fusion_stage.layers.0.residual_layer1.batch_norm2.num_batches_tracked', 'neck.fusion_stage.layers.0.residual_layer1.convolution1.weight', 'neck.fusion_stage.layers.0.residual_layer1.batch_norm1.num_batches_tracked', 'neck.fusion_stage.layers.0.residual_layer1.batch_norm2.

In [15]:
!CUDA_VISIBLE_DEVICES=0,1 ucc-train -c  ../configs/public/unet/base.yml \
                                    -o  global.find_lr=False \
                                        global.wandb=True \
                                        data.args.train.root_dir='../data' \
                                        data.args.val.root_dir='../data' \
                                        trainer.learning_rate=0.036 \
                                        trainer.num_epochs=2 #notebook only

Overriding configurating
Seed set to 1940
/home/nhtlong/mambaforge/envs/ucc2/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/nhtlong/mambaforge/envs/ucc2/bin/ucc-train -c  ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
wandb: Currently logged in as: nhtlongcs (ucc-quest-23). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.1
wandb: Run data is saved locally in ./runs/wandb/run-20231215_205457-xrmx5r53
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run baseline-model-lr-2023_12_15-20_54_56
wandb: ⭐️ View project at https://wandb.ai/ucc-quest-23/ucc-quest-23-public
wandb: 🚀 View run at https://wandb.ai/ucc-quest

In [5]:
import os 
import os.path as osp
import shutil 

RUNS_FOLDER='/home/nhtlong/DCU/ucc-seg/segmentation/runs/'
PROJECT_NAME='ucc-quest-23-public-lb'
RUN_ID='3tjf1aku'
CKPT_DIR=osp.join(RUNS_FOLDER, PROJECT_NAME, RUN_ID, 'checkpoints')
CKPT_PATH=osp.join(RUNS_FOLDER, PROJECT_NAME, RUN_ID, 'checkpoints', 'best.ckpt')

best_ckpt = os.listdir(CKPT_DIR)
best_ckpt.sort()
best_ckpt = best_ckpt[-1]
shutil.copy(osp.join(CKPT_DIR, best_ckpt), CKPT_PATH)
CKPT_PATH, best_ckpt

('/home/nhtlong/DCU/ucc-seg/segmentation/runs/ucc-quest-23-public-lb/3tjf1aku/checkpoints/best.ckpt',
 'dpt-ade-DPT-epoch=24-val_high_vegetation__IoU=84.2900.ckpt')

In [5]:
%%time
# EVAL WITH ORIGINAL SIZE
!ucc-eval   -c  ../configs/public/tf/segformerb5-ade.yml \
            -o  global.pretrained=$CKPT_PATH \
                data.args.train.root_dir='../data' \
                data.args.val.root_dir='../data'

Overriding configurating
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b3-finetuned-ade-512-512 and are newly initialized because the shapes did not match:
- decode_head.classifier.weight: found shape torch.Size([150, 768, 1, 1]) in the checkpoint and torch.Size([2, 768, 1, 1]) in the model instantiated
- decode_head.classifier.bias: found shape torch.Size([150]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b5-finetuned-ade-640-640 and are newly initialized because the shapes did not match:
- decode_head.classifier.weight: found shape torch.Size([150, 768, 1, 1]) in the checkpoint and torch.Size([2, 768, 1, 1]) in the model instantiated
- decode_head.classifier.bias: found sh

In [6]:
# EVAL WITH RESIZED IMGS
!ucc-eval-fast  -c  ../configs/public/tf/dpt-ade.yml \
                -o  global.pretrained=$CKPT_PATH \
                    data.args.train.root_dir='../data' \
                    data.args.val.root_dir='../data'

Overriding configurating
Some weights of DPTForSemanticSegmentation were not initialized from the model checkpoint at Intel/dpt-large-ade and are newly initialized: ['neck.fusion_stage.layers.0.residual_layer1.convolution1.weight', 'neck.fusion_stage.layers.0.residual_layer1.batch_norm1.running_var', 'neck.fusion_stage.layers.0.residual_layer1.batch_norm1.num_batches_tracked', 'neck.fusion_stage.layers.0.residual_layer1.batch_norm2.weight', 'neck.fusion_stage.layers.0.residual_layer1.batch_norm1.bias', 'neck.fusion_stage.layers.0.residual_layer1.batch_norm1.running_mean', 'neck.fusion_stage.layers.0.residual_layer1.batch_norm2.running_var', 'neck.fusion_stage.layers.0.residual_layer1.batch_norm2.num_batches_tracked', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.weight', 'neck.fusion_stage.layers.0.residual_layer1.batch_norm2.bias', 'neck.fusion_stage.layers.0.residual_layer1.batch_norm2.running_mean', 'neck.fusion_stage.layers.0.residual_layer1.batch_norm1.weight']
You shou

In [9]:
!mkdir -p ./tmp/valid/

In [10]:
!cat ../configs/public/tf/dpt-ade.yml | grep -E "model|name|PRETRAINED|NUM_CLASS" 

    username: ucc-quest-23
    project_name: "ucc-quest-23-public-lb"
    name: "dpt-ade-tunelr" # experiment name
    name: SegDataset
model:
    name: DPT
        NUM_CLASS: 2
        PRETRAINED: null
    - name: SMAPIoUMetricWrapper
    - name: ModelCheckpoint
          filename: "{epoch}-{val_high_vegetation__IoU:.4f}"
    - name: EarlyStopping
    - name: LearningRateMonitor
    - name: SemanticVisualizerCallbackWanDB


In [14]:
%%writefile ./tmp/valid/inference.yml
global:
    pretrained: null
    save_path: null
    batch_size: 1
data:
    SIZE: 380 # ORIGINAL SIZE
    IMG_DIR: null
model:
    name: DPT
    args:
        NUM_CLASS: 2
        PRETRAINED: null

Overwriting ./tmp/valid/inference.yml


In [15]:
!ucc-pred   -c  ./tmp/valid/inference.yml \
            -o  global.pretrained=$CKPT_PATH \
                data.IMG_DIR=../data/public/img/valid/ \
                global.save_path=./tmp/valid/results.json

Overriding configurating
Some weights of DPTForSemanticSegmentation were not initialized from the model checkpoint at Intel/dpt-large-ade and are newly initialized: ['neck.fusion_stage.layers.0.residual_layer1.batch_norm2.running_var', 'neck.fusion_stage.layers.0.residual_layer1.convolution1.weight', 'neck.fusion_stage.layers.0.residual_layer1.batch_norm1.bias', 'neck.fusion_stage.layers.0.residual_layer1.batch_norm2.num_batches_tracked', 'neck.fusion_stage.layers.0.residual_layer1.batch_norm1.num_batches_tracked', 'neck.fusion_stage.layers.0.residual_layer1.batch_norm2.weight', 'neck.fusion_stage.layers.0.residual_layer1.batch_norm1.running_var', 'neck.fusion_stage.layers.0.residual_layer1.batch_norm1.running_mean', 'neck.fusion_stage.layers.0.residual_layer1.batch_norm1.weight', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.weight', 'neck.fusion_stage.layers.0.residual_layer1.batch_norm2.running_mean', 'neck.fusion_stage.layers.0.residual_layer1.batch_norm2.bias']
You shou